# Training a Single Model for Computer Vision and Natural Language Processing

In this notebook, we train a single MANN model to perform both a computer vision task (image classification) as well as a natural language processing task (sentiment analysis).  For this notebook, we will use the MNIST Fashion dataset as well as the IMDB Sentiment Analysis dataset to train and test on.  Training will be done in a sequential manner, i.e. the network will first learn to perform image classification and then will learn to perform sentiment analysis.

In [1]:
# Load the packages required for the experiment
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
import pandas as pd
import numpy as np
import mann

2022-01-04 14:23:03.476937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-04 14:23:03.476971: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Load and Preprocess the Data

Our first step in this demonstration will be to load the data.  We are using the TensorFlow functions to download and/or load the datasets into memory.

The `Fashion MNIST` dataset, which can be found at [this link](https://github.com/zalandoresearch/fashion-mnist), contains 60,000 grayscale training images of clothing items from 10 different classes, each 28 pixels high and 28 pixels wide.  The test set contains 10,000 grayscale images from the same 10 classes and with the same shape.  The only preprocessing we will be doing is dividing all pixel values by 255 to ensure all values fall in the interval [0, 1].

The `IMDB Large Movie Review` dataset, which can be found at [this link](https://ai.stanford.edu/~amaas/data/sentiment/), contains 25,000 "highly polar" movie reviews for training and 25,000 reviews for testing.  When loaded through the TensorFlow function, these reviews are already converted into sequences of integers, where each integer corresponds to an individual word.  For our purposes, we will restrict the vocabulary to 10,000 words, so only the top 10,000 words will be represented, and any other words will be left as "out of vocabulary" (OOV).  Once that is complete, we will preprocess the inputs so that only the first 500 words in each review are considered, truncated and padded from the rear of the review.

In [2]:
# Load the data
(fashion_x_train, fashion_y_train), (fashion_x_test, fashion_y_test) = tf.keras.datasets.fashion_mnist.load_data()
(imdb_x_train, imdb_y_train), (imdb_x_test, imdb_y_test) = tf.keras.datasets.imdb.load_data(num_words = 10000)

# Preprocess each of the input datasets. For the images, normalize each of the pixels to values between 0 and 1.
# For the reviews, truncate and/or pad the lengths to 500 words each
fashion_x_train = fashion_x_train/255
fashion_x_test = fashion_x_test/255
imdb_x_train = tf.keras.preprocessing.sequence.pad_sequences(imdb_x_train, maxlen = 500, padding = 'post', truncating = 'post')
imdb_x_test = tf.keras.preprocessing.sequence.pad_sequences(imdb_x_test, maxlen = 500, padding = 'post', truncating = 'post')

# Reshape the target data to having one column
fashion_y_train = fashion_y_train.reshape(-1, 1)
fashion_y_test = fashion_y_test.reshape(-1, 1)
imdb_y_train = imdb_y_train.reshape(-1, 1)
imdb_y_test = imdb_y_test.reshape(-1, 1)

## Create the Model

Now that the data has been loaded and preprocessed, we can create the model.  Because of the different input shapes for our inputs, we have to develop some dedicated input regions within the model.  After data flows through those regions, which are pruned along with the rest of the network and include only a flatten and single fully connected layer for the fashion data and an embedding and fully connected layer for the IMDB data, the bulk of the network consists of shared layers.  

In these shared layers, the training procedure identifies disjoint subsets of weights to dedicate to each individual task.  By isolating these highly-pruned subnetworks and representing the network weights across an extra, task-specific dimension, the network is able to learn multiple tasks in parallel or in sequence without exhibiting any interference between tasks.

After passing through the shared layers, there are dedicated output layers for each task.  These are required because of the different output shapes between tasks - the image classification task differentiates between 10 different classes, whereas the sentiment analysis task performs binary classification to identify positive and negative reviews.

After the model is instantiated, we utilize the `mask_model` function to perform masking (aka pruning) of the model.  This identifies the isolated subnetworks to perform each individual task and prepares the network for multitask training.

In [3]:
# Create the input block for the fashion data, which includes an input layer, a flatten layer, and a masked dense layer
fashion_input = tf.keras.layers.Input(fashion_x_train.shape[1:])
fashion_flatten = tf.keras.layers.Flatten()(fashion_input)
fashion_reshape = mann.layers.MaskedDense(512, activation = 'relu')(fashion_flatten)

# Create the input block for the reviews data, which includes an input layer, an embedding layer, a flatten layer,
# and a masked dense layer of equal output shape to the masked dense layer for the fashion input block
imdb_input = tf.keras.layers.Input(imdb_x_train.shape[1:])
imdb_embedding = tf.keras.layers.Embedding(10000, 2)(imdb_input)
imdb_flatten = tf.keras.layers.Flatten()(imdb_embedding)
imdb_reshape = mann.layers.MaskedDense(512, activation = 'relu')(imdb_flatten)

# Now that the shapes align for each of the tasks, we can push the data through multitask layers
x = mann.layers.MultiMaskedDense(256, activation = 'relu')([fashion_reshape, imdb_reshape])
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)
x = mann.layers.MultiMaskedDense(256, activation = 'relu')(x)

# Output block for the fashion data
fashion_selector = mann.layers.SelectorLayer(0)(x)
fashion_output = mann.layers.MaskedDense(10, activation = 'softmax')(fashion_selector)

# Output block for the IMDB data
imdb_selector = mann.layers.SelectorLayer(1)(x)
imdb_output = mann.layers.MaskedDense(1, activation = 'sigmoid')(imdb_selector)

# Instantiate the model and compile it
model = tf.keras.models.Model([fashion_input, imdb_input], [fashion_output, imdb_output])
model.compile(
    loss = ['sparse_categorical_crossentropy', 'binary_crossentropy'],
    metrics = 'accuracy',
    optimizer = 'adam'
)

# Mask (prune) the model and recompile for training
model = mann.utils.mask_model(
    model,
    90,
    x = [fashion_x_train[:1000], imdb_x_train[:1000]],
    y = [fashion_y_train[:1000], imdb_y_train[:1000]]
)
model.compile(
    loss = ['sparse_categorical_crossentropy', 'binary_crossentropy'],
    metrics = 'accuracy',
    optimizer = 'adam'
)
model.summary()

2022-01-04 14:23:09.582403: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-04 14:23:09.582716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-04 14:23:09.582773: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-01-04 14:23:09.582803: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-01-04 14:23:09.582831: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 2)       20000       ['input_2[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 784)          0           ['input_1[0][0]']                
                                                                                              

## Train the Model on the First Task

After the model has been prepared, we can train it.  As stated previously, we are training the model in sequence on the individual tasks.  To do so, we compile the model so as to only optimize the loss for the specific task that is currently being trained, then pass dummy data to the task which is not being trained (in this case, empty-valued arrays).  The pruning previously done ensures only task-specific weights are being trained.

In [4]:
# This cell compiles the model for training task 1 (fashion) and trains the model for that task
callback = tf.keras.callbacks.EarlyStopping(min_delta = 0.01, patience = 3, restore_best_weights = True)
model.compile(
    loss = ['sparse_categorical_crossentropy', 'binary_crossentropy'],
    metrics = 'accuracy',
    optimizer = 'adam',
    loss_weights = [1, 0]
)
model.fit(
    [fashion_x_train, np.zeros((fashion_x_train.shape[0], imdb_x_train.shape[1]))],
    [fashion_y_train, np.zeros(fashion_y_train.shape)],
    batch_size = 512,
    epochs = 100,
    callbacks = [callback],
    validation_split = 0.2,
    verbose = 2
)

Epoch 1/100
94/94 - 4s - loss: 1.8025 - masked_dense_2_loss: 1.8025 - masked_dense_3_loss: 0.6932 - masked_dense_2_accuracy: 0.2560 - masked_dense_3_accuracy: 0.0000e+00 - val_loss: 1.1106 - val_masked_dense_2_loss: 1.1106 - val_masked_dense_3_loss: 0.6932 - val_masked_dense_2_accuracy: 0.4996 - val_masked_dense_3_accuracy: 0.0000e+00 - 4s/epoch - 40ms/step
Epoch 2/100
94/94 - 2s - loss: 1.0035 - masked_dense_2_loss: 1.0035 - masked_dense_3_loss: 0.6932 - masked_dense_2_accuracy: 0.5629 - masked_dense_3_accuracy: 0.0000e+00 - val_loss: 0.9581 - val_masked_dense_2_loss: 0.9581 - val_masked_dense_3_loss: 0.6932 - val_masked_dense_2_accuracy: 0.5941 - val_masked_dense_3_accuracy: 0.0000e+00 - 2s/epoch - 24ms/step
Epoch 3/100
94/94 - 2s - loss: 0.9133 - masked_dense_2_loss: 0.9133 - masked_dense_3_loss: 0.6932 - masked_dense_2_accuracy: 0.6152 - masked_dense_3_accuracy: 0.0000e+00 - val_loss: 0.8694 - val_masked_dense_2_loss: 0.8694 - val_masked_dense_3_loss: 0.6932 - val_masked_dense_2_ac

# Train the Model on the Second Task

Once the model has been trained on the first task, we repeat the process to train on the second task, this time compiling the model such that only the second task is trained for.

In [5]:
# This cell compiles the model for training task 2 (IMDB) and trains the model for that task
model.compile(
    loss = ['sparse_categorical_crossentropy', 'binary_crossentropy'],
    metrics = 'accuracy',
    optimizer = 'adam',
    loss_weights = [0, 1]
)
model.fit(
    [np.zeros((imdb_x_train.shape[0],) + fashion_x_train.shape[1:]), imdb_x_train],
    [np.zeros(imdb_y_train.shape[0]), imdb_y_train],
    batch_size = 128,
    epochs = 100,
    callbacks = [callback],
    validation_split = 0.2,
    verbose = 2
)

Epoch 1/100
157/157 - 4s - loss: 0.6932 - masked_dense_2_loss: 2.1084 - masked_dense_3_loss: 0.6932 - masked_dense_2_accuracy: 0.0000e+00 - masked_dense_3_accuracy: 0.5016 - val_loss: 0.6932 - val_masked_dense_2_loss: 2.1084 - val_masked_dense_3_loss: 0.6932 - val_masked_dense_2_accuracy: 0.0000e+00 - val_masked_dense_3_accuracy: 0.4938 - 4s/epoch - 22ms/step
Epoch 2/100
157/157 - 2s - loss: 0.6569 - masked_dense_2_loss: 2.1084 - masked_dense_3_loss: 0.6569 - masked_dense_2_accuracy: 0.0000e+00 - masked_dense_3_accuracy: 0.5861 - val_loss: 0.5202 - val_masked_dense_2_loss: 2.1084 - val_masked_dense_3_loss: 0.5202 - val_masked_dense_2_accuracy: 0.0000e+00 - val_masked_dense_3_accuracy: 0.8172 - 2s/epoch - 14ms/step
Epoch 3/100
157/157 - 2s - loss: 0.3223 - masked_dense_2_loss: 2.1084 - masked_dense_3_loss: 0.3223 - masked_dense_2_accuracy: 0.0000e+00 - masked_dense_3_accuracy: 0.8741 - val_loss: 0.3091 - val_masked_dense_2_loss: 2.1084 - val_masked_dense_3_loss: 0.3091 - val_masked_dens

## Remove masks

After training is complete, we can perform the optional `remove_layer_masks` function, which removes the extra masking tensors utilized only during the training process.  This removal reduces the number of weights within the network, thus further optimizing it for prediction.

In [6]:
simplified_model = mann.utils.remove_layer_masks(model)
simplified_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 2)       20000       ['input_2[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 784)          0           ['input_1[0][0]']                
                                                                                              

## Get predictions and Report Performance

Lastly, we retrieve the predictions on the test data from the model and report the performance of the model on each of the individual tasks.

In [7]:
fashion_preds = simplified_model.predict([fashion_x_test, np.zeros((fashion_x_test.shape[0], imdb_x_test.shape[1]))])[0].argmax(axis = 1)
imdb_preds = (simplified_model.predict([np.zeros((imdb_x_test.shape[0],) + fashion_x_test.shape[1:]), imdb_x_test])[1].flatten() >= 0.5).astype(int)

## Fashion Performance

In [8]:
print('Fashion Test Performance:')
print('\n')
print(confusion_matrix(fashion_y_test, fashion_preds))
print('\n')
print(classification_report(fashion_y_test, fashion_preds))

Fashion Test Performance:


[[809   3  44  47   1   1  86   0   9   0]
 [  1 959   3  32   2   0   2   0   1   0]
 [ 11   0 743  13 149   0  80   0   4   0]
 [ 20  10  17 891  40   0  18   0   4   0]
 [  0   2  94  39 813   1  47   0   4   0]
 [  0   0   0   0   0 954   0  25   4  17]
 [174   2 146  38 119   0 502   0  19   0]
 [  0   0   0   0   0  22   0 953   0  25]
 [  0   1   3   4   9  12  21   3 945   2]
 [  0   0   0   0   0  21   0  49   2 928]]


              precision    recall  f1-score   support

           0       0.80      0.81      0.80      1000
           1       0.98      0.96      0.97      1000
           2       0.71      0.74      0.72      1000
           3       0.84      0.89      0.86      1000
           4       0.72      0.81      0.76      1000
           5       0.94      0.95      0.95      1000
           6       0.66      0.50      0.57      1000
           7       0.93      0.95      0.94      1000
           8       0.95      0.94      0.95      100

## IMDB Performance

In [9]:
print('IMDB Test Performance:')
print('\n')
print((confusion_matrix(imdb_y_test, imdb_preds)))
print('\n')
print(classification_report(imdb_y_test, imdb_preds))

IMDB Test Performance:


[[10768  1732]
 [ 1734 10766]]


              precision    recall  f1-score   support

           0       0.86      0.86      0.86     12500
           1       0.86      0.86      0.86     12500

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000



## Save and load the model

An important part of being able to deploy models is the ability to save and load the models to disk.  We provide extra utilities to help save and load these models consistent with the `keras` API.

In [10]:
simplified_model.save('cv_and_nlp_model.h5')
loaded_model = tf.keras.models.load_model('cv_and_nlp_model.h5', custom_objects = mann.utils.get_custom_objects())
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 28, 28)]     0           []                               
                                                                                                  
 embedding (Embedding)          (None, 500, 2)       20000       ['input_2[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 784)          0           ['input_1[0][0]']                
                                                                                              